In [9]:
import pandas as pd
import json
import numpy as np

In [4]:
with open('../src/base_config.json') as json_file:
    config_dict1 = json.load(json_file)

with open('../src/period_config.json') as json_file:
    config_dict2 = json.load(json_file)

# Merging the two config dictionaries
config_dict = {**config_dict1, **config_dict2}


In [5]:
config_dict

{'POB_type_classifier': {'list_IR': ['IR', 'IR-NA', 'LFB'],
  'list_service': ['CR', 'CR-NA'],
  'list_deferred': ['RR', 'RR-NA'],
  'list_hybrid': ['BNDL']},
 'hybrid_pct_IR': 0.17,
 'list_columns': ['recognized_',
  'service_',
  'deferred_1M_',
  'deferred_3M_',
  'deferred_6M_',
  'deferred_1Y_',
  'deferred_2Y_',
  'deferred_3Y_'],
 'new_columns': ['fwd_01M',
  'fwd_02M',
  'fwd_03M',
  'fwd_04M',
  'fwd_05M',
  'fwd_06M',
  'fwd_07M',
  'fwd_08M',
  'fwd_09M',
  'fwd_10M',
  'fwd_11M',
  'fwd_12M'],
 'sales_doc_type': {'immediate_revenue': ['L2',
   'RE',
   'G2',
   'KE',
   'KR',
   'OR',
   'ZACR',
   'ZCCR',
   'ZCDR',
   'ZCOR',
   'ZCSR',
   'ZCRE',
   'ZCS2',
   'ZLO3',
   'ZLO1',
   'ZLO2',
   'ZOG2',
   'ZOL2',
   'ZOTM'],
  'service': ['ZATS', 'ZPS'],
  'deferred': ['ZCC',
   'ZCS1',
   'ZCQ',
   'ZCQA',
   'ZLCR',
   'ZLDR',
   'ZLG2',
   'ZLO5',
   'ZREB',
   'ZRG2',
   'ZRL2',
   'ZCSB']},
 'type_A_config_keepers': ['MTHLY', '1Y', '2Y', '3Y'],
 'type_D_classification

In [10]:
def load_ADBE_cal(config_dict):
    '''
    This function opens the Adobe financial calendar and creates a dataframe containing the information
    in the Adobe financial calendar that is needed for the deferred revenue model.
    The dataframe creates returns has two columns
        - The period, in YYYY-MM format
        - The number of weeks within th eperiod
    of weeks in each fiscal period (month)
    :param config_dict: the main configuration dictionary for the deferred revenue model
    :return: df: A dataframe containing all of the details of the Adobe financial calendar
    '''

    ADBE_cal_filename  = config_dict['ADBE_cal']['direct_filename']
    ADBE_cal_sheetname = config_dict['ADBE_cal']['sheetname']

    df = pd.read_excel(ADBE_cal_filename, ADBE_cal_sheetname)
    df["Period_Weeks"] = (df["Per_End"] - df["Per_Start"]) / np.timedelta64(
        1, "W"
    )
    df["Period_Weeks"] = df["Period_Weeks"].astype(int)
    df["Period_Weeks"] = df["Period_Weeks"] + 1

    # Creating a column in df_cal with year  '-' the last two digits of the per_ticker to match with the billings dataframe
    df["p2digit"] = df["Period"].astype(str)
    df["p2digit"] = df["p2digit"].str.zfill(2)

    df["period_match"] = (
            df["Year"].astype(str) + "-" + df["p2digit"].astype(str)
    )

    df.drop(["p2digit"], axis=1, inplace=True)

    df.drop(
        [
            "Year",
            "Quarter",
            "Period",
            "Qtr_Ticker",
            "Qtr_Start",
            "Qtr_End",
            "Per_Start",
            "Per_Ticker",
            "Per_End",
        ],
        axis=1,
        inplace=True,
    )

    return df


In [11]:
df = load_ADBE_cal(config_dict)

In [12]:
df

,Period_Weeks,period_match
0,5,2010-01
1,4,2010-02
2,5,2010-03
3,4,2010-04
4,4,2010-05
...,...,...
247,4,2030-08
248,5,2030-09
249,4,2030-10
250,4,2030-11


In [13]:
def load_FX_data(config_dict):
    '''
    This function takes the config dictionary and creates a dataframe from the FX_data.xlsx file.
    The final dataframe contains
    :param config_dict: 
    :return: 
    '''
    filename_FX = config_dict['path_to_data'] + config_dict['FX_rates']['filename']
    df_FX_rates = pd.read_excel(filename_FX, sheet_name=config_dict['FX_rates']['sheetname'])
    # some of the currencies do not contain forwards, so just filling across (as if no points)
    df_FX_rates = df_FX_rates.fillna(method='ffill', axis=1)
    df_FX_rates["VOL_3M"] = df_FX_rates["VOL_3M"] / 100
    df_FX_rates["VOL_6M"] = df_FX_rates["VOL_6M"] / 100
    df_FX_rates["VOL_9M"] = df_FX_rates["VOL_9M"] / 100
    df_FX_rates["VOL_1Y"] = df_FX_rates["VOL_1Y"] / 100

    df_FX_rates.head(5)
    return df_FX_rates


In [14]:
df_FX = load_FX_data(config_dict)
df_FX

,DC,Ticker,Spot,FWD_3M,FWD_6M,FWD_9M,FWD_1Y,VOL_3M,VOL_6M,VOL_9M,VOL_1Y
0,ARS,USDARS,74.175,87.6209,99.1677,111.233,103.231,0.2503,0.26905,0.278987,0.288925
1,AED,USDAED,3.67295,3.67365,3.6744,3.67548,3.6769,0.0106,0.014475,0.0174125,0.02035
2,AUD,AUDUSD,0.73895,0.739125,0.739213,0.739175,0.739092,0.1047,0.10135,0.10075,0.10015
3,BHD,USDBHD,0.377,0.37737,0.3777,0.377958,0.378245,0.00378245,0.00378245,0.00378245,0.00378245
4,BRL,USDBRL,5.46845,5.48472,5.49925,5.51932,5.54703,0.20855,0.192525,0.1855,0.178475
5,CAD,USDCAD,1.30225,1.30197,1.3016,1.30171,1.30194,0.0693,0.0678,0.0674,0.067
6,CHF,USDCHF,0.90315,0.900782,0.898115,0.895803,0.893545,0.0774,0.07405,0.07335,0.07265
7,CLP,USDCLP,776.655,776.055,775.145,774.525,774.205,0.134,0.12855,0.127875,0.1272
8,COP,USDCOP,3741.05,3763.95,3783.3,3806.05,3832.8,0.132925,0.1307,0.12865,0.1266
9,DKK,USDDKK,6.2345,6.22327,6.20874,6.1955,6.18139,0.0789,0.07625,0.07555,0.07485


In [15]:
def load_FX_fwds(config_dict):
    filename_FX_fwds = config_dict['path_to_data'] + config_dict['FX_forwards']['filename']
    FX_fwds_sheetname  = config_dict['FX_forwards']['sheetname']
    df_FX_fwds = pd.read_excel(
        filename_FX_fwds, sheet_name=FX_fwds_sheetname, skiprows=1, usecols="C,G",
    )

    df_FX_fwds.rename(
        index=str, columns={"Unnamed: 2": "curr", "FWD REF": "forward"}, inplace=True
    )
    return df_FX_fwds


In [16]:
df_FX_fwds = load_FX_fwds(config_dict)

In [17]:
df_FX_fwds


,curr,forward
0,AED,3.673748
1,AUD,1.401502
2,BBD,2.043700
3,BMD,1.000000
4,BRL,5.127479
5,CAD,1.341938
6,CHF,0.927770
7,CLP,768.327500
8,COP,3667.633800
9,CNH,7.044660


In [18]:
def load_curr_map(config_dict):
    filename_curr_map = config_dict['path_to_data'] + config_dict['curr_map']['filename']
    curr_map_sheetname = config_dict['curr_map']['sheetname']
    df_curr_map = pd.read_excel(filename_curr_map, sheet_name=curr_map_sheetname)
    df_curr_map["Country"] = df_curr_map["Country"].str.replace(
        "\(MA\)", "", case=False
    )
    df_curr_map["Country"] = df_curr_map["Country"].str.strip()

    return df_curr_map


In [19]:
df_curr_map = load_curr_map(config_dict)

In [20]:
df_curr_map

,Country,Currency
0,Aus and New Zealand,AUD
1,Benelux,EUR
2,Brazil,USD
3,Canada,USD
4,China,USD
5,Eastern Europe,EUR
6,France,EUR
7,Germany,EUR
8,Hong Kong & Taiwan,USD
9,Iberica,EUR
